In [1]:
# using python 3 to better handle unicode emojis
import emoji
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows',500)

%matplotlib inline

# Goal: 
Using   the   training   data   in    ages_train.csv ,   build   a   statistical   model   to   predict   the
age   of   users   in    ages_test.csv,     based   on   any   of   the   information   provided   (profiles, friend   networks,   and   mentions).

# Load data

In [2]:
# load json files: 
ap = pd.read_json('assignment_package/age_profiles.json')
at = pd.read_json('assignment_package/age_tweets.json')
# fp = pd.read_json('assignment_package/friend_profiles.json')
# mp = pd.read_json('assignment_package/mention_profiles.json')

In [4]:
# load csv files:
ages_train = pd.read_csv('assignment_package/ages_train.csv')
mentions = pd.read_csv('assignment_package/mentions.csv')
friends = pd.read_csv('assignment_package/friends.csv')

In [6]:
mentions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17695 entries, 0 to 17694
Data columns (total 3 columns):
ID                 17695 non-null int64
MentionedID        17695 non-null int64
MentionedHandle    17695 non-null object
dtypes: int64(2), object(1)
memory usage: 414.8+ KB


In [7]:
mentions.head()

,ID,MentionedID,MentionedHandle
0,612815277,798747864,AlenaBacalia
1,612815277,354590068,hfholden34
2,612815277,401111769,MarilynYousif
3,612815277,35581752,heeeeyerin
4,612815277,1591956090,DogeTheDog


In [8]:
friends.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46609 entries, 0 to 46608
Data columns (total 2 columns):
ID          46609 non-null int64
FriendID    46609 non-null int64
dtypes: int64(2)
memory usage: 728.3 KB


In [9]:
friends.head()

,ID,FriendID
0,612815277,574147452
1,612815277,36767299
2,612815277,29374598
3,612815277,67634814
4,612815277,30008146


In [10]:
ages_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1711 entries, 0 to 1710
Data columns (total 2 columns):
ID     1711 non-null int64
Age    1711 non-null int64
dtypes: int64(2)
memory usage: 26.8 KB


In [11]:
ages_train.head()

,ID,Age
0,23605717,21
1,781084760,19
2,102262916,21
3,413710015,21
4,584304916,90
